In [7]:
!git clone https://github.com/sathishkumar67/PODFCSSV.git
!mv PODFCSSV/* /kaggle/working/

Cloning into 'PODFCSSV'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 72 (delta 18), reused 55 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (72/72), 893.18 KiB | 17.51 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [5]:
import os
import shutil

current_path = os.getcwd()

for item in os.listdir(current_path):
    item_path = os.path.join(current_path, item)

    if os.path.isfile(item_path) or os.path.islink(item_path):
        os.remove(item_path)
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)

In [8]:
!python data/prepare_data.py

Initializing Data Split: 2 Clients, 2 Tasks (50 classes/task), Alpha=0.5
100%|████████████████████████████████████████| 169M/169M [00:04<00:00, 42.2MB/s]
Processing Task 0 (Classes 0-49)...
Processing Task 1 (Classes 50-99)...
✅ Data preparation complete. Splits saved to ./data/federated_splits.json

--- Split Statistics ---
Client 0:
  Task 0: 14378 samples (Classes 0-49)
  Task 1: 13103 samples (Classes 50-99)
Client 1:
  Task 0: 10622 samples (Classes 0-49)
  Task 1: 11897 samples (Classes 50-99)
